# Python for Finance - part 11

#### 금융분석과 관련된 기본적인 내용을 정리하였습니다. 파이썬을 활용한 금융분석 (이브 힐피시 지음)을 참고하였습니다.

## Chapter 18 포트폴리오 가치 평가
### 18.1 파생상품 포지션

In [1]:
class derivatives_position(object):
    """ 파생상품 포지션 모형 클래스 """
    
    def __init__(self, name, quantity, underlying, mar_env, otype, payoff_func):
        self.name = name
        self.quantity = quantity
        self.underlying = underlying
        self.mar_env = mar_env
        self.otype = otype
        self.payoff_func = payoff_func
        
    def get_info(self):
        print("NAME")
        print(self.name, '\n')
        print("QUANTITY")
        print(self.quantity, '\n')
        print("UNDERLYING")
        print(self.underlying, '\n')
        print("MARKET ENVIRONMENT")
        print(self.mar_env, '\n')
        print("\n**Constants**")
        for key, value in self.mar_env.constatns.items():
            print(key, value)
        print("\n**Lists**")
        for key, value in self.mar_env.lists.items():
            print(key, value)
        print("\n**Curves**")
        for key, value in self.mar_env.curves.items():
            print(key, value)
        print("OPTION TYPE")
        print(self.otype, '\n')
        print("PAYOFF FUNCTION")
        print(self.payoff_func, '\n')

### 18.2 파생상품 포트폴리오

In [2]:
import numpy as np
import pandas as pd

models = {'gbm' : geometric_brownian_motion,
          'jd' : jump_diffusion,
          'srd' : square_root_diffusion}
otypes = {'European' : valuation_mcs_european,
          'American' : valuation_mcs_american}

class derivatives_portfolio(object):
    """ 파생상품 포지션 포트폴리오 생성용 클래스 """
    
    def __init__(self, name, positions, val_env, assets, correlations=None, fixed_seed=False):
        self.name = name
        self.positions = positions
        self.val_env = val_env
        self.assets = assets
        self.underlyings = set()
        self.correlations = correlations
        self.time_grid = None
        self.underlying_objects = {}
        self.valuation_objects = {}
        self.fixed_seed = fixed_seed
        self.special_dates = []
        
        for pos in self.positions:
            self.val_env.constants['starting_date'] = min(self.val_env.constants['starting_date'], position[pos].mar_env.pricing_date)
            self.val_env.constants['final_date'] = max(self.val_env.constants['final_date'], position[pos].mar_env.constants['maturity'])
            self.underlyings.add(positions[pos].underlying)
            
        start = self.val_env.constants['starting_date']
        end = self.val_env.constants['final_date']
        time_grid = pd.date_range(start= start, end=end, freq=self.val_env.constants['frequency'])
        time_grid = list(time_grid)
        
        for pos in self.positions:
            maturity_date = positions[pos].mar_env.constants['maturity']
            if maturity_date not in time_grid:
                time_grid.insert(0, maturity_date)
                self.special_dates.append(maturity_date)
                
        if start not in time_grid:
            time_grid.append(end)
            
        time_grid = list(set(time_grid))
        time_grid.sort()
        self.time_grid = np.array(time_grid)
        self.val_env.add_list('time_grid', self.time_grid)
        
        if correlations is not None:
            ul_list = sorted(self.underlyings)
            correlation_matrix = np.zeros((len(len(ul_list), len(ul_list))))
            np.fill_diagonal(correlation_matrix, 1.)
            correlation_matrix = pd.DataFrame(correlation_matrix, index=ul_list, columns=ul_list)
            
            for i, j, corr in correlations:
                corr = min(corr, 0.99999999999999)
                correlation_matrix.loc[i,j] = corr
                correlation_matrix.loc[j, i] = corr
                
            cholesky_matrix = np.linalg.cholesky(np.array(correlation_matrix))
            
            rn_set = {aseet: ul_list.index(asset) for asset in self.underlyings}
            
            random_numbers = sn_random_numbers((len(rn_set), len(self.time_grid), self.val_env.constants['paths']), fixed_seed=self.fixed_seed)
            
            self.val_env.add_list('cholesky_matrix', cholesky_matrix)
            self.val_env.add_list('random_numbers', random_numbers)
            self.val_env.add_list('rn_set', rn_set)
            
        for aseet in self.underlyings:
            mar_env = self.assets[asset]
            mar_env.add_environment(val_env)
            model = models[mar_env.constants['model']]
            
            if correlations is not None:
                self.underlying_objects[asset] = model(asset, mar_env, corr=True)
                
            else:
                self.underlying_objects[asset] = model(asset, mar_env, corr=False)
                
        for pos in positions:
            val_class = otypes[positions[pos].otype]
            mar_env  = positions[pos].mar_env
            mar_env.add_environment(self.val_env)
            self.valuation_objects[pos] = val_class(name=positions[pos].name, mar_env=mar_env, underlying=self.underlying_objects[positions[pos].underlying], payoff_func=positions[pos].payoff_func)
            
            
    def get_positions(self):
        """ 포트폴리오 내의 모든 파생상품 포지션에 대한 정보를 얻기 위한 메서드 """
        
        for pos in self.positions:
            bar = '\n' + 50*'-'
            print(bar)
            self.positions[pos].get_info()
            print(bar)
            
    
    def get_statistics(self, fixed_seed=False):
        """ 포트폴리오 통계 제공 """
        res_list = []
        
        for pos, value in self.valuation_objects.items():
            p = self.positions[pos]
            pv = value.present_value(fixed_seed=fixed_seed)
            res_list.append([p.name, p.quantity, pv, value.currency, pv * p.quantity, value.delta() * p.quantity, value.vega() * p.quantity,])
            
        res_df = pd.DataFrame(res_list, columns = ['name', 'quant', 'value', 'curr.', 'pos_value', 'pos_delta', 'pos_vega'])
        
        return res_df

NameError: name 'geometric_brownian_motion' is not defined